### STEP 1 - Import the necessary packages

In [ ]:
# STEP 1 - Import the necessary packages

import ollama
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display


In [4]:
# STEP 2: Basic configuration parameters for the ollama API

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type":"application/json"}
MODEL = "llama3.2"

In [ ]:
# STEP 3: Send a message to the model and the model response is displayed in markdown
# I am simply testing the model with a simple message

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = ollama.chat(model=MODEL, messages=[{"role":"user", "content":message}])
display(Markdown(response.message.content))

Welcome! It's great to meet you for the first time. I'm excited to chat with you and help answer any questions or topics you'd like to discuss. I'm here to provide information, assistance, and a friendly conversation.

To get us started, how can I help you today? Is there something specific on your mind that you'd like to talk about, or do you just want to see where the conversation takes us?

In [ ]:
# STEP 4: Create a class that will be used to represent a website 
# we are using BeautifulSoup to parse the website and remove the irrelevant tags 

# some websites need you to use proper header when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers) # get the response from the url
        soup = BeautifulSoup(response.content, 'html.parser') # parse the response content
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]): # remove the script, style, img, and input tags
            irrelevant.decompose() # remove the tags
        self.text = soup.body.get_text(separator="\n", strip=True) # what is left is the text of the website

# what is BeautifulSoup?
# BeautifulSoup is a library that allows you to parse HTML and XML documents.
# It provides a simple interface for navigating, searching, and modifying the parse tree.
# It is a powerful tool for web scraping and data extraction.  

In [ ]:
# STEP 5: let's scrape the website

ed = Website("https://edwarddonner.com") # create a website object
print(ed.title)
print(ed.text)


Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [ ]:
# STEP 8: define the system prompt
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown"

In [ ]:
# STEP 9: A function to create the user prompt incorporating the website's title and text

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt



In [43]:
# STEP 10: let's see how the 
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

In [44]:
# STEP 11: This function to create the messages for the ollama API
# it includes the system prompt and the user prompt for the website
# REMEMBER:
# the system prompt is the prompt that the model will use to understand the website
# the user prompt is the prompt that the model will use to summarize the website
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [45]:
# This is how the message we are sending to the ollama API
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a 

In [46]:
# STEP 12: This function is used to summarize a website
# it creates a website object and then calls the ollama API with the messages for the website
# it returns the response from the ollama API

def summarize(url):
    website = Website(url)
    response = ollama.chat(model=MODEL, messages=messages_for(website))
    return response.message.content

In [ ]:
# STEP 13: This function is called to summarize a website
# it returns the response from the ollama API in markdown format
summarize(ed.url)

'**Summary**\n\nThis website belongs to Edward Donner, a co-founder and CTO of Nebula.io. He is an AI enthusiast who writes about his experiences with Large Language Models (LLMs) and shares updates on his projects.\n\n### News/Announcements\n\n* **Courses and Workshops**: Upcoming courses include "Connecting my courses" and "The Complete Agentic AI Engineering Course", as well as a LLM Workshop.\n* **Events**: Announcements for 2025 AI Executive Briefings, taking place on April 21st.\n* **New Product Release**: No specific mention, but Edward mentions that Nebula.io\'s award-winning platform has press coverage and happy customers.\n\n### General Information\n\nThe website also includes links to social media profiles, a newsletter subscription form, and contact information (ed [at] edwarddonner [dot] com).'

In [48]:
# STEP 14: This function displays the website summary nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [49]:
# This function calls the summarize function and displays the summary in markdown format
display_summary(ed.url)

**Summary**
================

* The website is run by Ed, the co-founder and CTO of Nebula.io, an AI startup that applies AI to help people discover their potential.
* The company's product is used by recruiters to source, understand, engage, and manage talent.
* Ed also runs online courses on LLMs (Large Language Models) and has a newsletter.

**News/Announcements**
=====================

* **May 28, 2025**: "Connecting my courses – become an LLM expert and leader"
* **May 18, 2025**: "2025 AI Executive Briefing" (no further details available)
* **April 21, 2025**: No further details available
* **January 23, 2025**: "The Complete Agentic AI Engineering Course" (no further details available)
* **LLM Workshop – Hands-on with Agents – resources** (no date or details available)

In [ ]:
# Another website to summarize

display_summary("https://investors.com")

This is the homepage of Investor's Business Daily (IBD), a financial news organization that provides stock lists, market data, and research to help investors make more money in the stock market.

The page features various tools and resources for investors, including:

1. **Stock Lists**: IBD offers several stock lists, including:
	* IBD 50: A list of top stocks showing strong price action and earnings growth.
	* Sector Rotation: A list of sectors that are experiencing rotation and potential upside.
	* Leaderboard: A model portfolio of the best 10-15 stocks to buy.
2. **Market Data**: IBD provides real-time stock quotes, charts, and news to help investors stay up-to-date on market performance.
3. **Research and Analysis**: IBD offers in-depth research reports, analysis, and commentary on individual stocks and sectors.
4. **Premium Tools**: IBD offers several premium tools for advanced investors, including:
	* MarketSurge: A tool that helps investors find undervalued stocks with strong growth potential.
	* SwingTrader: A tool that provides trade ideas and market analysis.
	* IBD Digital: A digital platform that provides access to IBD's research reports, analysis, and tools.
5. **News and Commentary**: IBD features news articles, commentary, and analysis on various financial topics.

Overall, the page is designed to provide investors with the information and resources they need to make informed investment decisions.

**Key Features**

* Real-time stock quotes and charts
* Research reports and analysis on individual stocks and sectors
* Model portfolio and trade ideas through Leaderboard and SwingTrader
* Premium tools for advanced investors
* News and commentary on various financial topics

**Target Audience**

* Individual investors looking to make more money in the stock market.
* Active traders seeking trade ideas and market analysis.
* Advanced investors interested in premium tools and research reports.

**Tone and Style**

* Informative and educational, with a focus on helping investors make informed decisions.
* Engaging and accessible, with clear explanations of complex financial concepts.
* Objective and unbiased, with no recommendations or solicitations to buy or sell securities.